In [1]:
### Some initializations
import numpy as np
import math
import random
import copy
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import lingam
import pickle
import warnings
from scipy.stats import binomtest
import itertools
import loli
from heapq import nlargest
from matplotlib.patches import Patch
from sklearn.cluster import KMeans

In [9]:
import pickle
from scipy.stats import binomtest
import numpy as np
E=300
n=30
with open('LorenzResults='+str(E)+'N'+str(n)+'.pkl','rb') as f:
    Lorenz30=pickle.load(f)

final=np.zeros((6,6))
for tar in range(6):
    for key in Lorenz30[tar]:
        for var in range(6):
            if var in key:
                final[tar,var]+=Lorenz30[tar][key]

print(final)
pvals=np.zeros((6,6))
for i in range(6):
    for j in range(6):
        test=binomtest(np.int32(final[i,j]),500,0.1,alternative='greater')
        pvals[i,j]=test.pvalue
print(pvals<0.05)

[[490. 132.  16.   4.  12.   9.]
 [ 54. 402. 357.  67.  86. 120.]
 [ 74. 452. 462.  32.  35.  48.]
 [  6.   8.   6. 495. 192.  12.]
 [102. 104. 164.  64. 327. 317.]
 [ 32.  59.  51. 101. 443. 443.]]
[[ True  True False False False False]
 [False  True  True  True  True  True]
 [ True  True  True False False False]
 [False False False  True  True False]
 [ True  True  True  True  True  True]
 [False False False  True  True  True]]


In [8]:
binomtest(54,500,0.1,alternative='greater')

BinomTestResult(k=54, n=500, alternative='greater', statistic=0.108, pvalue=0.29593284067935044)

In [2]:
# Via Stack Overflow
# https://stackoverflow.com/questions/11130156/suppress-stdout-stderr-print-from-python-functions
# Supressing the output of annyoing libraries
from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull

@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)

In [3]:
def yerr(tot,res):
    yerrbar=np.zeros((2,len(res)))
    for i in range(len(res)):

        if res[i]>1:
            res[i]=1
        if res[i]<0:
            res[i]=0
        result = binomtest(k=int(res[i]*tot), n=tot, p=res[i])
        yerrbar[0,i]=res[i]-result.proportion_ci()[0]
        yerrbar[1,i]=result.proportion_ci()[1]-res[i]
    return yerrbar

In [8]:
### Changing sample size
np.random.seed(1)
d=6  #Dimensionality
supp=(1,2) #support indices
s=len(supp)  #Number of support entries
sample=[8,16,24,32]
fntrunc=np.zeros((len(sample)))
fptrunc=np.zeros((len(sample)))
fnt=np.zeros((len(sample)))
fpt=np.zeros((len(sample)))
fnlingtrunc=np.zeros((len(sample)))
fplingtrunc=np.zeros((len(sample)))
fnlingt=np.zeros((len(sample)))
fplingt=np.zeros((len(sample)))
runs=300
nmax=sample[-1]
l=1 # Number of intervals we combine for the statistics
B=100 # Bootstrap runs
I=30      #Number of Intervals 
a=[list(itertools.combinations(range(d), k)) for k in range(0,d+1)]
subsets = [item for sublist in a for item in sublist]
dic={}
# We first fix the data for all runs, to eliminate the randomness of this.
betas=np.zeros((d))
x_uni=np.zeros((runs,I,nmax,d))
x_t=np.zeros((runs,I,nmax,d))
y_uni=np.zeros((runs,I,nmax))
y_t=np.zeros((runs,I,nmax))
for r in range(runs):
    for i in range(I):
        betas[np.array(supp)]=np.random.uniform(low=1,high=5,size=(s))
        std=np.random.uniform(low=1,high=5,size=(d))
        x_uni[r,i,:,0]=np.random.uniform(low=-6*std[0]**2,high=6*std[0]**2,size=(nmax))
        x_uni[r,i,:,1]=x_uni[r,i,:,0]+np.random.uniform(low=-6*std[1]**2,high=6*std[1]**2,size=(nmax))
        x_uni[r,i,:,2]=0.3*x_uni[r,i,:,1]+np.random.uniform(low=-6*std[2]**2,high=6*std[2]**2,size=(nmax))
        x_uni[r,i,:,3]=0.2*x_uni[r,i,:,2]+np.random.uniform(low=-6*std[3]**2,high=6*std[3]**2,size=(nmax))
        e=np.random.uniform(low=-6*2**2,high=6*2**2,size=(nmax))
        y_uni[r,i,:]=x_uni[r,i,:,:]@betas+e
        x_uni[r,i,:,4]=np.random.uniform(low=-6*std[4]**2,high=6*std[4]**2,size=(nmax))+0.1*x_uni[r,i,:,1]+y_uni[r,i,:]
        x_uni[r,i,:,5]=np.random.uniform(low=-6*std[4]**2,high=6*std[4]**2,size=(nmax))+y_uni[r,i,:]
        
        
        x_t[r,i,:,0]=np.random.standard_t(2*(std[0])**(2)/((std[0])**(2)-1),size=(nmax))
        x_t[r,i,:,1]=x_t[r,i,:,0]+np.random.standard_t(2*(std[1])**(2)/((std[1])**(2)-1),size=(nmax))
        x_t[r,i,:,2]=0.3*x_t[r,i,:,1]+np.random.standard_t(2*(std[2])**(2)/((std[2])**(2)-1),size=(nmax))
        x_t[r,i,:,3]=0.2*x_t[r,i,:,2]+np.random.standard_t(2*(std[3])**(2)/((std[3])**(2)-1),size=(nmax))
        e=np.random.standard_t(2*(2)**(2)/((2)**(2)-1),size=(nmax))
        y_t[r,i,:]=x_t[r,i,:]@betas+e
        x_t[r,i,:,4]=np.random.standard_t(2*(std[4])**(2)/((std[4])**(2)-1),size=(nmax))+0.1*x_t[r,i,:,1]+y_t[r,i,:]
        x_t[r,i,:,5]=np.random.standard_t(2*(std[5])**(2)/((std[5])**(2)-1),size=(nmax))+y_t[r,i,:]
            
            
for o,n in enumerate(sample):
    print('Sample size',o+1,'of',len(sample))
    for r in range(runs):
        Xt=[]
        Yt=[]
        Xtrunc=[]
        Ytrunc=[]
        X_lingtrunc=[]
        X_lingt=[]
        for i in range(I):
            x_lingtrunc=np.zeros((n,d+1))
            x_lingt=np.zeros((n,d+1))
            
            Xtrunc.append(x_uni[r,i,:n,:])
            Ytrunc.append(y_uni[r,i,:n])
            
            x_lingtrunc[:,:d]=copy.copy(x_uni[r,i,:n,:])
            x_lingtrunc[:,d]=copy.copy(y_uni[r,i,:n])
            
            X_lingtrunc.append(x_lingtrunc)


            Xt.append(x_t[r,i,:n,:])
            Yt.append(y_t[r,i,:n])
            x_lingt[:,:d]=copy.copy(x_t[r,i,:n,:])
            x_lingt[:,d]=copy.copy(y_t[r,i,:n])
            X_lingt.append(x_lingt)
            
        with suppress_stdout_stderr():
            model=lingam.MultiGroupDirectLiNGAM()
            model.fit(X_lingt)

            
        plausibleS=loli.gauss(Xt,Yt,alpha=0.1)
        if not not plausibleS:
            supphat=set.intersection(*plausibleS)
            if len(supphat.difference(set(supp)))>0:
                fpt[o]+=1/runs
            if len(set(supp).difference(supphat))>0:
                fnt[o]+=1/runs
                
        lingmat=model.adjacency_matrices_[0][6,:]
        lingsupp=np.where(lingmat!=0)
        supphatling=set(lingsupp[0])
        if len(supphatling.difference(set(supp)))>0:
            fplingt[o]+=1/runs
        if len(set(supp).difference(supphatling))>0:
            fnlingt[o]+=1/runs
            
        with suppress_stdout_stderr():
            model=lingam.MultiGroupDirectLiNGAM()
            model.fit(X_lingtrunc)

            
        plausibleS=loli.gauss(Xtrunc,Ytrunc,alpha=0.1)
        if not not plausibleS:
            supphat=set.intersection(*plausibleS)
            if len(supphat.difference(set(supp)))>0:
                fptrunc[o]+=1/runs
            if len(set(supp).difference(supphat))>0:
                fntrunc[o]+=1/runs
                
        lingmat=model.adjacency_matrices_[0][6,:]
        lingsupp=np.where(lingmat!=0)
        supphatling=set(lingsupp[0])
        if len(supphatling.difference(set(supp)))>0:
            fplingtrunc[o]+=1/runs
        if len(set(supp).difference(supphatling))>0:
            fnlingtrunc[o]+=1/runs
            


ComparisonLingSample={}
ComparisonLingSample['fpt']=fpt
ComparisonLingSample['fnt']=fnt
ComparisonLingSample['fptrunc']=fptrunc
ComparisonLingSample['fntrunc']=fntrunc
ComparisonLingSample['fplingt']=fplingt
ComparisonLingSample['fnlingt']=fnlingt
ComparisonLingSample['fnlingtrunc']=fnlingtrunc
ComparisonLingSample['fplingtrunc']=fplingtrunc
ComparisonLingSample['sample']=sample

with open('ComparisonLingSample.pkl', 'wb') as f:
    pickle.dump(ComparisonLingSample, f)

Sample size 1 of 4
Sample size 2 of 4
Sample size 3 of 4
Sample size 4 of 4


In [ ]:
### Changing heterogeneity
np.random.seed(1)
d=6  #Dimensionality
supp=(1,2) #support indices
s=len(supp)  #Number of support entries
hetero=[0,0.5,1,1.5,2]
fntrunc=np.zeros((len(hetero)))
fptrunc=np.zeros((len(hetero)))
fnt=np.zeros((len(hetero)))
fpt=np.zeros((len(hetero)))
fnlingtrunc=np.zeros((len(hetero)))
fplingtrunc=np.zeros((len(hetero)))
fnlingt=np.zeros((len(hetero)))
fplingt=np.zeros((len(hetero)))
runs=300
n=20
l=1 # Number of intervals we combine for the statistics
B=100 # Bootstrap runs
I=30      #Number of Intervals 
a=[list(itertools.combinations(range(d), k)) for k in range(0,d+1)]
subsets = [item for sublist in a for item in sublist]
dic={}

betas=np.zeros((runs,I,d))

for o,h in enumerate(hetero):
    print('Heterogeneity',o+1,'of',len(hetero))
    for r in range(runs):
        Xt=[]
        Yt=[]
        Xtrunc=[]
        Ytrunc=[]
        X_lingtrunc=[]
        X_lingt=[]
        for i in range(I):
            betas[r,i,supp]=np.random.uniform(low=1,high=1+h,size=(s))
            x=np.zeros((n,d))
            x_lingtrunc=np.zeros((n,d+1))
            x_lingt=np.zeros((n,d+1))
            y=np.zeros((n))
            std=np.random.uniform(low=2,high=2+h,size=(d))
            x[:,0]=np.random.uniform(low=-6*std[0]**2,high=6*std[0]**2,size=(n))
            x[:,1]=x[:,0]+np.random.uniform(low=-6*std[1]**2,high=6*std[1]**2,size=(n))
            x[:,2]=0.3*x[:,1]+np.random.uniform(low=-6*std[2]**2,high=6*std[2]**2,size=(n))
            x[:,3]=0.2*x[:,2]+np.random.uniform(low=-6*std[3]**2,high=6*std[3]**2,size=(n))
            e=np.random.uniform(low=-6*2**2,high=6*2**2,size=(n))
            y=x@betas[r,i,:]+e
            x[:,4]=np.random.uniform(low=-6*std[4]**2,high=6*std[4]**2,size=(n))+0.1*x[:,1]+y
            x[:,5]=np.random.uniform(low=-6*std[4]**2,high=6*std[4]**2,size=(n))+y

            Xtrunc.append(x)
            Ytrunc.append(y)
            x_lingtrunc[:,:d]=copy.copy(x)
            x_lingtrunc[:,d]=copy.copy(y)
            X_lingtrunc.append(x_lingtrunc)
            
            x=np.zeros((n,d))
            y=np.zeros((n))
            std=np.random.uniform(low=2,high=2+h,size=(d))
            x[:,0]=np.random.standard_t(2*(std[0])**(2)/((std[0])**(2)-1),size=(n))
            x[:,1]=x[:,0]+np.random.standard_t(2*(std[1])**(2)/((std[1])**(2)-1),size=(n))
            x[:,2]=0.3*x[:,1]+np.random.standard_t(2*(std[2])**(2)/((std[2])**(2)-1),size=(n))
            x[:,3]=0.2*x[:,2]+np.random.standard_t(2*(std[3])**(2)/((std[3])**(2)-1),size=(n))
            e=np.random.standard_t(2*(2)**(2)/((2)**(2)-1),size=(n))
            y=x@betas[r,i,:]+e
            x[:,4]=np.random.standard_t(2*(std[4])**(2)/((std[4])**(2)-1),size=(n))+0.1*x[:,1]+y
            x[:,5]=np.random.standard_t(2*(std[5])**(2)/((std[5])**(2)-1),size=(n))+y

            Xt.append(x)
            Yt.append(y)
            x_lingt[:,:d]=copy.copy(x)
            x_lingt[:,d]=copy.copy(y)
            X_lingt.append(x_lingt)
            
        with suppress_stdout_stderr():
            model=lingam.MultiGroupDirectLiNGAM()
            model.fit(X_lingt)

            
        plausibleS=loli.gauss(Xt,Yt,alpha=0.1)
        if not not plausibleS:
            supphat=set.intersection(*plausibleS)
            if len(supphat.difference(set(supp)))>0:
                fpt[o]+=1/runs
            if len(set(supp).difference(supphat))>0:
                fnt[o]+=1/runs
                
        lingmat=model.adjacency_matrices_[0][6,:]
        lingsupp=np.where(lingmat!=0)
        supphatling=set(lingsupp[0])
        if len(supphatling.difference(set(supp)))>0:
            fplingt[o]+=1/runs
        if len(set(supp).difference(supphatling))>0:
            fnlingt[o]+=1/runs
            
        with suppress_stdout_stderr():
            model=lingam.MultiGroupDirectLiNGAM()
            model.fit(X_lingtrunc)

            
        plausibleS=loli.gauss(Xtrunc,Ytrunc,alpha=0.1)
        if not not plausibleS:
            supphat=set.intersection(*plausibleS)
            if len(supphat.difference(set(supp)))>0:
                fptrunc[o]+=1/runs
            if len(set(supp).difference(supphat))>0:
                fntrunc[o]+=1/runs
                
        lingmat=model.adjacency_matrices_[0][6,:]
        lingsupp=np.where(lingmat!=0)
        supphatling=set(lingsupp[0])
        if len(supphatling.difference(set(supp)))>0:
            fplingtrunc[o]+=1/runs
        if len(set(supp).difference(supphatling))>0:
            fnlingtrunc[o]+=1/runs
        

ComparisonLingHetero={}
ComparisonLingHetero['fpt']=fpt
ComparisonLingHetero['fnt']=fnt
ComparisonLingHetero['fptrunc']=fptrunc
ComparisonLingHetero['fntrunc']=fntrunc
ComparisonLingHetero['fplingt']=fplingt
ComparisonLingHetero['fnlingt']=fnlingt
ComparisonLingHetero['fnlingtrunc']=fnlingtrunc
ComparisonLingHetero['fplingtrunc']=fplingtrunc
ComparisonLingHetero['hetero']=hetero

with open('ComparisonLingHetero2.pkl', 'wb') as f:
    pickle.dump(ComparisonLingHetero, f)

In [4]:
### Changing number of environments
np.random.seed(1)
d=6  #Dimensionality
supp=(1,2) #support indices
s=len(supp)  #Number of support entries
for n in range[16,24]:
    environments=[30,50,80,100,150,200]
    # environments=[150,200]
    fnt=np.zeros((len(environments)))
    fpt=np.zeros((len(environments)))
    fnlingt=np.zeros((len(environments)))
    fplingt=np.zeros((len(environments)))
    runs=300
    l=1 # Number of intervals we combine for the statistics
    B=100 # Bootstrap runs
    Imax=environments[-1]
    a=[list(itertools.combinations(range(d), k)) for k in range(0,d+1)]
    subsets = [item for sublist in a for item in sublist]
    dic={}
    # We first fix the data for all runs, to eliminate the randomness of this.
    betas=np.zeros((d))
    x_uni=np.zeros((runs,Imax,n,d))
    x_t=np.zeros((runs,Imax,n,d))
    y_uni=np.zeros((runs,Imax,n))
    y_t=np.zeros((runs,Imax,n))
    for r in range(runs):
        for i in range(Imax):
            betas[np.array(supp)]=np.random.uniform(low=1,high=5,size=(s))
            std=np.random.uniform(low=1,high=5,size=(d))

            x_t[r,i,:,0]=np.random.standard_t(2*(std[0])**(2)/((std[0])**(2)-1),size=(n))
            x_t[r,i,:,1]=x_t[r,i,:,0]+np.random.standard_t(2*(std[1])**(2)/((std[1])**(2)-1),size=(n))
            x_t[r,i,:,2]=0.3*x_t[r,i,:,1]+np.random.standard_t(2*(std[2])**(2)/((std[2])**(2)-1),size=(n))
            x_t[r,i,:,3]=0.2*x_t[r,i,:,2]+np.random.standard_t(2*(std[3])**(2)/((std[3])**(2)-1),size=(n))
            e=np.random.standard_t(2*(2)**(2)/((2)**(2)-1),size=(n))
            y_t[r,i,:]=x_t[r,i,:]@betas+e
            x_t[r,i,:,4]=np.random.standard_t(2*(std[4])**(2)/((std[4])**(2)-1),size=(n))+0.1*x_t[r,i,:,1]+y_t[r,i,:]
            x_t[r,i,:,5]=np.random.standard_t(2*(std[5])**(2)/((std[5])**(2)-1),size=(n))+y_t[r,i,:]


    for o,I in enumerate(environments):
        print('Environments',o+1,'of',len(environments))
        for r in range(runs):
            Xt=[]
            Yt=[]
            X_lingt=[]




            for i in range(I):
                x_lingt=np.zeros((n,d+1))
                x=x_t[r,i,:,:]
                y=y_t[r,i,:]
                Xt.append(x)
                Yt.append(y)
                x_lingt[:,:d]=copy.copy(x)
                x_lingt[:,d]=copy.copy(y)
                X_lingt.append(x_lingt)

            with suppress_stdout_stderr():
                model=lingam.MultiGroupDirectLiNGAM()
                model.fit(X_lingt)


            plausibleS=loli.gauss(Xt,Yt,alpha=0.1)
            if not not plausibleS:
                supphat=set.intersection(*plausibleS)
                if len(supphat.difference(set(supp)))>0:
                    fpt[o]+=1/runs
                if len(set(supp).difference(supphat))>0:
                    fnt[o]+=1/runs

            lingmat=model.adjacency_matrices_[0][6,:]
            lingsupp=np.where(lingmat!=0)
            supphatling=set(lingsupp[0])
            if len(supphatling.difference(set(supp)))>0:
                fplingt[o]+=1/runs
            if len(set(supp).difference(supphatling))>0:
                fnlingt[o]+=1/runs



    ComparisonLingEnvironments={}
    ComparisonLingEnvironments['fpt']=fpt
    ComparisonLingEnvironments['fnt']=fnt
    ComparisonLingEnvironments['fplingt']=fplingt
    ComparisonLingEnvironments['fnlingt']=fnlingt
    ComparisonLingEnvironments['environments']=environments

    with open('ComparisonLingEnvironmentsN='+str(n)+'.pkl', 'wb') as f:
        pickle.dump(ComparisonLingEnvironments, f)

Environments 1 of 6
Environments 2 of 6
Environments 3 of 6
Environments 4 of 6
Environments 5 of 6
Environments 6 of 6
